# 🤖 Fine-Tuning en NLP avec DistilBERT

## 📚 Introduction

### 🎯 Qu'est-ce que le Fine-Tuning en NLP ?

Le fine-tuning en NLP consiste à :

1. **Partir d'un modèle pré-entraîné** (comme DistilBERT entraîné sur des millions de textes)
2. **Ajouter une couche de classification** adaptée à notre tâche spécifique
3. **Réentraîner partiellement** le modèle sur nos données

### 🧠 Pourquoi DistilBERT ?

**DistilBERT** est une version "distillée" de BERT :

✅ **40% plus petit** que BERT original

✅ **60% plus rapide** à l'exécution

✅ **97% des performances** de BERT

✅ **Parfait pour l'apprentissage** et les applications en production

### 🎬 Notre Projet : Analyse de Sentiment

Dans ce notebook, nous allons :
- Analyser le sentiment de critiques de films (positif/négatif)
- Utiliser le dataset IMDB (Internet Movie Database)
- Appliquer le fine-tuning avec DistilBERT
- Visualiser et comprendre chaque étape du processus

---

## 🔧 Configuration et Imports

Commençons par importer toutes les bibliothèques nécessaires et configurer notre environnement NLP.

In [ ]:
# Imports principaux pour le NLP
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from transformers import TFAutoModel, AutoTokenizer
import warnings
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Configuration pour un meilleur affichage
warnings.filterwarnings('ignore')
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print(f"🚀 TensorFlow version: {tf.__version__}")
print(f"💻 GPU disponible: {'✅ Oui' if len(tf.config.list_physical_devices('GPU')) > 0 else '❌ Non (CPU utilisé)'}")

### 🖥️ Configuration GPU pour NLP

Les modèles de NLP comme DistilBERT consomment beaucoup de mémoire. Configurons le GPU prudemment :

In [ ]:
# Configuration GPU spéciale pour les modèles NLP volumineux
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Utiliser seulement le premier GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        # Croissance mémoire progressive (important pour les transformers)
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print(f"✅ Configuration GPU réussie: {gpus[0].name}")
        print("🧠 Mémoire GPU configurée en croissance progressive")
    except RuntimeError as e:
        print(f"⚠️ Erreur configuration GPU: {e}")
else:
    print("🔧 Utilisation du CPU - Les modèles NLP fonctionnent aussi (plus lentement)")

print("\n💡 Info: Les modèles de transformers comme DistilBERT utilisent beaucoup de mémoire")
print("   La croissance progressive évite les erreurs 'Out of Memory'")

### ⚙️ Paramètres d'Entraînement pour NLP

Les paramètres pour le fine-tuning en NLP sont différents de ceux en vision :

In [ ]:
# 📊 Paramètres spécifiques au fine-tuning NLP
BATCH_SIZE = 16            # Plus petit qu'en vision (modèles plus volumineux)
MAX_LENGTH = 256           # Longueur maximale des séquences (tokens)
EPOCHS = 3                 # Peu d'époques suffisent en fine-tuning NLP
LEARNING_RATE = 2e-5       # Learning rate typique pour BERT/DistilBERT
NUM_SAMPLES_TRAIN = 5000   # Limitation pour l'exemple pédagogique
NUM_SAMPLES_VAL = 1000     # Échantillon de validation

print("📋 Configuration d'entraînement NLP:")
print(f"   • Taille de batch: {BATCH_SIZE} (plus petit pour les transformers)")
print(f"   • Longueur max: {MAX_LENGTH} tokens")
print(f"   • Nombre d'époques: {EPOCHS} (convergence rapide en fine-tuning)")
print(f"   • Learning rate: {LEARNING_RATE} (optimal pour BERT)")
print(f"   • Échantillons train/val: {NUM_SAMPLES_TRAIN}/{NUM_SAMPLES_VAL}")

print("\n🎓 Différences avec la Computer Vision:")
print("   🔹 Batch size plus petit (modèles plus volumineux)")
print("   🔹 Moins d'époques (pas de surapprentissage)")
print("   🔹 Learning rate spécifique aux transformers")
print("   🔹 Gestion de séquences de longueur variable")

### 🧠 Comprendre le Fine-Tuning en NLP

Avant de commencer, comprenons bien ce que nous allons faire :

In [ ]:
# 🎨 Visualisation conceptuelle du fine-tuning NLP
def visualiser_fine_tuning_nlp():
    """
    Crée un diagramme explicatif du fine-tuning en NLP
    """
    fig, ax = plt.subplots(1, 1, figsize=(14, 10))
    
    # Définir les composants et leurs positions
    components = [
        {"name": "Texte d'entrée\n'This movie is great!'\n(Critique de film)", "pos": (1, 9), "color": "lightblue", "frozen": False},
        {"name": "Tokenizer\nDistilBERT\n[CLS] this movie is great [SEP]", "pos": (1, 7.5), "color": "lightyellow", "frozen": False},
        {"name": "DistilBERT\n(Pré-entraîné)\n🔒 PARTIELLEMENT GELÉ", "pos": (1, 6), "color": "lightcoral", "frozen": True},
        {"name": "Embeddings Contextuels\n768 dimensions\n(Représentation riche)", "pos": (1, 4.5), "color": "lightgreen", "frozen": True},
        {"name": "Pooling Layer\n[CLS] token\n→ Représentation globale", "pos": (1, 3), "color": "lightyellow", "frozen": False},
        {"name": "Dropout(0.3)\n🎲 Régularisation", "pos": (1, 1.5), "color": "lightpink", "frozen": False},
        {"name": "Dense(1)\n+ Sigmoid\n🎯 Classification Binaire", "pos": (1, 0), "color": "lightsteelblue", "frozen": False}
    ]
    
    # Dessiner les composants
    for i, comp in enumerate(components):
        x, y = comp["pos"]
        
        # Style différent pour les couches gelées
        if comp["frozen"]:
            bbox_props = dict(boxstyle="round,pad=0.3", facecolor=comp["color"], 
                            edgecolor="red", linewidth=2, linestyle="--")
        else:
            bbox_props = dict(boxstyle="round,pad=0.3", facecolor=comp["color"], 
                            edgecolor="black", linewidth=1)
        
        ax.text(x, y, comp["name"], ha="center", va="center", 
               fontsize=11, fontweight="bold", bbox=bbox_props)
        
        # Flèches entre composants
        if i < len(components) - 1:
            next_y = components[i+1]["pos"][1]
            ax.annotate("", xy=(x, next_y + 0.4), xytext=(x, y - 0.4),
                       arrowprops=dict(arrowstyle="->", lw=2, color="darkblue"))
    
    # Annotations explicatives
    ax.text(3.5, 8, "💭 Étape 1: Tokenisation\n• Conversion texte → tokens\n• Ajout tokens spéciaux [CLS], [SEP]\n• Padding/Truncation à 256 tokens", 
           fontsize=10, bbox=dict(boxstyle="round,pad=0.5", facecolor="wheat", alpha=0.8))
    
    ax.text(3.5, 5.5, "🧠 Étape 2: DistilBERT\n• Compréhension contextuelle\n• 66M paramètres pré-entraînés\n• Attention multi-têtes", 
           fontsize=10, bbox=dict(boxstyle="round,pad=0.5", facecolor="lightcyan", alpha=0.8))
    
    ax.text(3.5, 2, "🎯 Étape 3: Classification\n• Token [CLS] → représentation globale\n• Nouvelle tâche: sentiment\n• Apprentissage adaptatif", 
           fontsize=10, bbox=dict(boxstyle="round,pad=0.5", facecolor="lavender", alpha=0.8))
    
    # Configuration des axes
    ax.set_xlim(-0.5, 6)
    ax.set_ylim(-1, 10)
    ax.set_title("Architecture du Fine-Tuning NLP avec DistilBERT", fontsize=16, fontweight="bold", pad=20)
    ax.axis('off')
    
    # Légende
    legend_elements = [
        plt.Rectangle((0, 0), 1, 1, facecolor='lightcoral', edgecolor='red', linestyle='--', label='Couches pré-entraînées'),
        plt.Rectangle((0, 0), 1, 1, facecolor='lightsteelblue', edgecolor='black', label='Nouvelles couches')
    ]
    ax.legend(handles=legend_elements, loc='upper right')
    
    plt.tight_layout()
    plt.show()

print("🎨 Visualisation de l'architecture NLP:")
visualiser_fine_tuning_nlp()

print("\n💡 Points clés du fine-tuning NLP:")
print("   🧠 DistilBERT comprend déjà le langage (pré-entraîné)")
print("   🎯 On ajoute juste une tâche spécifique (classification sentiment)")
print("   ⚡ Convergence très rapide (2-3 époques)")
print("   🎲 Attention à la régularisation (modèles puissants)")

## 🎬 Chargement des Données - Dataset IMDB

Nous allons utiliser le fameux dataset **IMDB** (Internet Movie Database) qui contient des critiques de films avec leur sentiment associé.

### 📊 Qu'est-ce que le dataset IMDB ?

- **25,000 critiques** d'entraînement + 25,000 de test
- **2 classes** : Positif (👍) et Négatif (👎)
- **Textes de longueur variable** (quelques mots à plusieurs paragraphes)
- **Langue anglaise** avec vocabulaire riche et varié
- **Référence en NLP** pour l'analyse de sentiment

In [ ]:
# 🎬 Chargement du dataset IMDB
print("📥 Téléchargement du dataset IMDB...")
print("(Cela peut prendre quelques minutes la première fois)\n")

# Chargement avec TensorFlow Datasets
(train_data, test_data), info = tfds.load(
    'imdb_reviews',
    split=['train[:80%]', 'train[80%:]'],  # 80% train, 20% validation
    with_info=True,
    as_supervised=True  # Retourne (texte, label)
)

# 📊 Informations sur le dataset
print(f"✅ Dataset IMDB chargé avec succès !")
print(f"📈 Classes: {info.features['label'].names}")
print(f"📊 Total d'exemples: {info.splits['train'].num_examples}")
print(f"🔢 Distribution: Équilibrée (50% positif, 50% négatif)")

### 🔍 Conversion et Limitation des Données

Pour cet exemple pédagogique, nous allons limiter le nombre d'exemples pour un entraînement plus rapide :

In [ ]:
# 🔄 Conversion des données en listes Python
print("🔧 Préparation des données...")

# Extraction des textes et labels d'entraînement
print(f"📥 Extraction de {NUM_SAMPLES_TRAIN} exemples d'entraînement...")
train_texts = []
train_labels = []

for text, label in train_data.take(NUM_SAMPLES_TRAIN):
    train_texts.append(text.numpy().decode('utf-8'))
    train_labels.append(label.numpy())

# Extraction des textes et labels de validation
print(f"📥 Extraction de {NUM_SAMPLES_VAL} exemples de validation...")
test_texts = []
test_labels = []

for text, label in test_data.take(NUM_SAMPLES_VAL):
    test_texts.append(text.numpy().decode('utf-8'))
    test_labels.append(label.numpy())

# Conversion en arrays NumPy
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

print(f"\n✅ Données préparées:")
print(f"   📚 Entraînement: {len(train_texts)} exemples")
print(f"   🎯 Validation: {len(test_texts)} exemples")
print(f"   📊 Ratio train/val: {len(train_texts)/len(test_texts):.1f}:1")

### 📝 Exploration des Données

Regardons de plus près le contenu de notre dataset :

In [ ]:
# 🔍 Analyse des données
print("🔍 Analyse du contenu des données:\n")

# Distribution des classes
unique, counts = np.unique(train_labels, return_counts=True)
print("📊 Distribution des classes (entraînement):")
for class_id, count in zip(unique, counts):
    class_name = "Positif 👍" if class_id == 1 else "Négatif 👎"
    percentage = count / len(train_labels) * 100
    print(f"   {class_name}: {count} exemples ({percentage:.1f}%)")

# Statistiques sur la longueur des textes
text_lengths = [len(text.split()) for text in train_texts]
print(f"\n📏 Statistiques de longueur (en mots):")
print(f"   📈 Moyenne: {np.mean(text_lengths):.1f} mots")
print(f"   📊 Médiane: {np.median(text_lengths):.0f} mots")
print(f"   📉 Min: {min(text_lengths)} mots")
print(f"   📈 Max: {max(text_lengths)} mots")
print(f"   📊 75e percentile: {np.percentile(text_lengths, 75):.0f} mots")

### 📚 Exemples de Critiques

Examinons quelques exemples concrets pour mieux comprendre notre dataset :

In [ ]:
# 📝 Affichage d'exemples représentatifs
def afficher_exemples(texts, labels, num_exemples=4):
    """
    Affiche des exemples de critiques avec leur sentiment
    """
    print("📝 Exemples de critiques de films:\n")
    
    for i in range(min(num_exemples, len(texts))):
        # Informations sur l'exemple
        sentiment = "Positif 👍" if labels[i] == 1 else "Négatif 👎"
        text = texts[i]
        word_count = len(text.split())
        
        print(f"━━━ Exemple {i+1} ━━━")
        print(f"🏷️ Sentiment: {sentiment}")
        print(f"📏 Longueur: {word_count} mots")
        print(f"📄 Texte: {text[:300]}{'...' if len(text) > 300 else ''}")
        print()

# Afficher des exemples d'entraînement
afficher_exemples(train_texts, train_labels, 3)

print("💡 Observations:")
print("   • Les critiques varient énormément en longueur")
print("   • Le vocabulaire est riche et expressif")
print("   • Présence d'expressions idiomatiques et de sarcasme")
print("   • Certaines critiques sont très courtes, d'autres très détaillées")

### 📊 Visualisation de la Distribution des Longueurs

Créons des graphiques pour mieux comprendre nos données :

In [ ]:
# 📊 Visualisation des statistiques des données
def visualiser_donnees_imdb(texts, labels):
    """
    Crée des visualisations pour comprendre le dataset IMDB
    """
    # Calculs préliminaires
    text_lengths = [len(text.split()) for text in texts]
    char_lengths = [len(text) for text in texts]
    
    # Créer la figure avec plusieurs sous-graphiques
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('📊 Analyse du Dataset IMDB - Critiques de Films', fontsize=16, fontweight='bold')
    
    # 1. Distribution des classes
    unique, counts = np.unique(labels, return_counts=True)
    class_names = ['Négatif 👎', 'Positif 👍']
    colors = ['salmon', 'lightgreen']
    
    axes[0,0].pie(counts, labels=class_names, colors=colors, autopct='%1.1f%%', startangle=90)
    axes[0,0].set_title('Distribution des Sentiments', fontweight='bold')
    
    # 2. Distribution des longueurs (en mots)
    axes[0,1].hist(text_lengths, bins=50, color='skyblue', alpha=0.7, edgecolor='black')
    axes[0,1].axvline(np.mean(text_lengths), color='red', linestyle='--', label=f'Moyenne: {np.mean(text_lengths):.0f}')
    axes[0,1].axvline(MAX_LENGTH, color='orange', linestyle='--', label=f'Limite: {MAX_LENGTH}')
    axes[0,1].set_title('Distribution des Longueurs (mots)', fontweight='bold')
    axes[0,1].set_xlabel('Nombre de mots')
    axes[0,1].set_ylabel('Fréquence')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Box plot par sentiment
    pos_lengths = [text_lengths[i] for i, label in enumerate(labels) if label == 1]
    neg_lengths = [text_lengths[i] for i, label in enumerate(labels) if label == 0]
    
    axes[1,0].boxplot([neg_lengths, pos_lengths], labels=['Négatif 👎', 'Positif 👍'])
    axes[1,0].set_title('Longueur des Textes par Sentiment', fontweight='bold')
    axes[1,0].set_ylabel('Nombre de mots')
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. Distribution cumulative
    sorted_lengths = np.sort(text_lengths)
    y = np.arange(1, len(sorted_lengths) + 1) / len(sorted_lengths)
    axes[1,1].plot(sorted_lengths, y, 'b-', linewidth=2)
    axes[1,1].axvline(MAX_LENGTH, color='orange', linestyle='--', label=f'Limite: {MAX_LENGTH}')
    axes[1,1].set_title('Distribution Cumulative des Longueurs', fontweight='bold')
    axes[1,1].set_xlabel('Nombre de mots')
    axes[1,1].set_ylabel('Proportion cumulative')
    axes[1,1].grid(True, alpha=0.3)
    axes[1,1].legend()
    
    # Pourcentage de textes sous la limite
    under_limit = sum(1 for length in text_lengths if length <= MAX_LENGTH) / len(text_lengths) * 100
    axes[1,1].text(0.6, 0.2, f'{under_limit:.1f}% des textes\nsont ≤ {MAX_LENGTH} mots', 
                  transform=axes[1,1].transAxes, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # Statistiques importantes
    print(f"\n📊 Statistiques importantes:")
    print(f"   📏 {under_limit:.1f}% des textes sont ≤ {MAX_LENGTH} mots")
    print(f"   ✂️ {100-under_limit:.1f}% seront tronqués lors de la tokenisation")
    print(f"   📈 Longueur moyenne: {np.mean(text_lengths):.1f} mots")
    print(f"   📊 Écart-type: {np.std(text_lengths):.1f} mots")

# Créer les visualisations
print("📊 Création des visualisations du dataset IMDB:")
visualiser_donnees_imdb(train_texts, train_labels)

### 🔤 Analyse du Vocabulaire

Regardons la richesse du vocabulaire dans notre dataset :

In [ ]:
# 🔤 Analyse du vocabulaire
def analyser_vocabulaire(texts, top_n=20):
    """
    Analyse le vocabulaire du dataset
    """
    print("🔤 Analyse du vocabulaire:")
    
    # Compter tous les mots
    from collections import Counter
    import re
    
    all_words = []
    for text in texts:
        # Nettoyage simple et tokenisation
        words = re.findall(r'\b\w+\b', text.lower())
        all_words.extend(words)
    
    word_counts = Counter(all_words)
    unique_words = len(word_counts)
    total_words = len(all_words)
    
    print(f"\n📊 Statistiques du vocabulaire:")
    print(f"   🔢 Mots uniques: {unique_words:,}")
    print(f"   📚 Total de mots: {total_words:,}")
    print(f"   📊 Richesse lexicale: {unique_words/total_words:.3f}")
    
    # Top mots les plus fréquents
    print(f"\n🏆 Top {top_n} mots les plus fréquents:")
    for i, (word, count) in enumerate(word_counts.most_common(top_n), 1):
        percentage = count / total_words * 100
        print(f"   {i:2d}. '{word}': {count:,} occurrences ({percentage:.2f}%)")
    
    # Visualisation des fréquences
    plt.figure(figsize=(12, 6))
    
    # Graphique 1: Top mots
    plt.subplot(1, 2, 1)
    top_words = word_counts.most_common(15)
    words, counts = zip(*top_words)
    plt.barh(range(len(words)), counts, color='skyblue')
    plt.yticks(range(len(words)), words)
    plt.xlabel('Fréquence')
    plt.title(f'Top {len(words)} Mots les Plus Fréquents', fontweight='bold')
    plt.gca().invert_yaxis()
    
    # Graphique 2: Distribution des fréquences
    plt.subplot(1, 2, 2)
    frequencies = list(word_counts.values())
    plt.hist(frequencies, bins=50, color='lightcoral', alpha=0.7, edgecolor='black')
    plt.xlabel('Fréquence des mots')
    plt.ylabel('Nombre de mots')
    plt.title('Distribution des Fréquences', fontweight='bold')
    plt.yscale('log')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return word_counts

# Analyser le vocabulaire
vocab_stats = analyser_vocabulaire(train_texts)

print("\n💡 Observations sur le vocabulaire:")
print("   • Présence de mots très fréquents (articles, prépositions)")
print("   • Vocabulaire spécialisé du cinéma ('movie', 'film', 'plot')")
print("   • Mots expressifs pour les sentiments ('great', 'terrible', 'amazing')")
print("   • Distribution typique: peu de mots très fréquents, beaucoup de mots rares")

## 🔤 Tokenisation avec DistilBERT

La **tokenisation** est l'étape qui transforme le texte brut en tokens numériques que le modèle peut comprendre.

### 🧠 Qu'est-ce que la Tokenisation ?

La tokenisation :
- **Divise le texte** en unités plus petites (tokens)
- **Convertit les mots** en indices numériques
- **Ajoute des tokens spéciaux** ([CLS], [SEP], [PAD])
- **Gère la longueur** (padding/truncation)

### 🎯 Spécificités du Tokenizer DistilBERT

- **WordPiece tokenization** : Divise les mots rares en sous-mots
- **Vocabulaire de ~30k tokens** : Couvre la plupart des mots anglais
- **Tokens spéciaux** :
  - `[CLS]` : Début de séquence (classification)
  - `[SEP]` : Fin de séquence
  - `[PAD]` : Padding pour égaliser les longueurs
  - `[UNK]` : Mots inconnus

In [ ]:
# 🔤 Chargement du tokenizer DistilBERT
print("🔤 Chargement du tokenizer DistilBERT...")

# Chargement depuis Hugging Face
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

print("✅ Tokenizer DistilBERT chargé avec succès !")
print(f"📊 Taille du vocabulaire: {tokenizer.vocab_size:,} tokens")
print(f"🔤 Modèle: distilbert-base-uncased (anglais, minuscules)")

# Informations sur les tokens spéciaux
print(f"\n🎯 Tokens spéciaux:")
print(f"   🏁 CLS (début): '{tokenizer.cls_token}' (ID: {tokenizer.cls_token_id})")
print(f"   🔚 SEP (fin): '{tokenizer.sep_token}' (ID: {tokenizer.sep_token_id})")
print(f"   📝 PAD (padding): '{tokenizer.pad_token}' (ID: {tokenizer.pad_token_id})")
print(f"   ❓ UNK (inconnu): '{tokenizer.unk_token}' (ID: {tokenizer.unk_token_id})")

### 🧪 Démonstration de la Tokenisation

Voyons comment le tokenizer transforme du texte en nombres :

In [ ]:
# 🧪 Démonstration de la tokenisation
def demonstrer_tokenisation(tokenizer, exemples_textes):
    """
    Démontre le processus de tokenisation étape par étape
    """
    print("🧪 === DÉMONSTRATION DE LA TOKENISATION ===")
    
    for i, texte in enumerate(exemples_textes):
        print(f"\n📝 Exemple {i+1}: '{texte}'")
        print("─" * 60)
        
        # Étape 1: Tokenisation basique
        tokens = tokenizer.tokenize(texte)
        print(f"🔤 Tokens: {tokens}")
        print(f"📊 Nombre de tokens: {len(tokens)}")
        
        # Étape 2: Conversion en IDs
        token_ids = tokenizer.convert_tokens_to_ids(tokens)
        print(f"🔢 IDs des tokens: {token_ids}")
        
        # Étape 3: Encoding complet (avec tokens spéciaux)
        encoded = tokenizer.encode(texte, add_special_tokens=True)
        print(f"🎯 Encodage complet: {encoded}")
        
        # Étape 4: Décodage pour vérifier
        decoded = tokenizer.decode(encoded)
        print(f"🔄 Décodage: '{decoded}'")
        
        # Explication des ajouts
        if len(encoded) > len(token_ids):
            print(f"➕ Tokens ajoutés: [CLS] au début, [SEP] à la fin")

# Exemples variés pour la démonstration
exemples = [
    "This movie is great!",
    "Absolutely terrible film",
    "The cinematography was breathtaking"
]

demonstrer_tokenisation(tokenizer, exemples)

### ⚙️ Fonction de Tokenisation pour nos Données

Créons la fonction qui va tokeniser tous nos textes :

In [ ]:
# ⚙️ Fonction de tokenisation pour l'entraînement
def tokenize_texts(texts, tokenizer, max_length=MAX_LENGTH):
    """
    Tokenise une liste de textes avec padding et truncation
    
    Args:
        texts: Liste de textes à tokeniser
        tokenizer: Tokenizer DistilBERT
        max_length: Longueur maximale des séquences
    
    Returns:
        Dict contenant input_ids et attention_mask
    """
    return tokenizer(
        texts,
        padding=True,              # Ajouter du padding pour égaliser les longueurs
        truncation=True,           # Tronquer les textes trop longs
        max_length=max_length,     # Longueur maximale
        return_tensors='tf'        # Retourner des tenseurs TensorFlow
    )

print("⚙️ Fonction de tokenisation définie")
print(f"\n🎯 Paramètres de tokenisation:")
print(f"   📏 Longueur maximale: {MAX_LENGTH} tokens")
print(f"   📝 Padding: Activé (égalise toutes les séquences)")
print(f"   ✂️ Truncation: Activée (coupe les textes trop longs)")
print(f"   🔧 Format de sortie: Tenseurs TensorFlow")

### 🔄 Tokenisation de nos Données

Appliquons maintenant la tokenisation à tous nos textes :

In [ ]:
# 🔄 Tokenisation des données d'entraînement et de validation
print("🔄 Tokenisation des données...\n")

# Tokenisation des données d'entraînement
print(f"📚 Tokenisation de {len(train_texts)} textes d'entraînement...")
train_encodings = tokenize_texts(train_texts, tokenizer)
print("✅ Entraînement tokenisé")

# Tokenisation des données de validation
print(f"🎯 Tokenisation de {len(test_texts)} textes de validation...")
test_encodings = tokenize_texts(test_texts, tokenizer)
print("✅ Validation tokenisée")

print(f"\n📊 Résultats de la tokenisation:")
print(f"   📈 Train - input_ids shape: {train_encodings['input_ids'].shape}")
print(f"   📈 Train - attention_mask shape: {train_encodings['attention_mask'].shape}")
print(f"   📊 Test - input_ids shape: {test_encodings['input_ids'].shape}")
print(f"   📊 Test - attention_mask shape: {test_encodings['attention_mask'].shape}")

print(f"\n💡 Explication des sorties:")
print(f"   🔢 input_ids: Les tokens convertis en nombres")
print(f"   👁️ attention_mask: Masque pour ignorer le padding (1=vrai token, 0=padding)")

### 🔍 Analyse de la Tokenisation

Examinons le résultat de la tokenisation sur quelques exemples :

In [ ]:
# 🔍 Analyse détaillée de la tokenisation
def analyser_tokenisation(texts, encodings, tokenizer, num_exemples=3):
    """
    Analyse détaillée du résultat de la tokenisation
    """
    print("🔍 === ANALYSE DÉTAILLÉE DE LA TOKENISATION ===")
    
    for i in range(min(num_exemples, len(texts))):
        print(f"\n📝 Exemple {i+1}:")
        print("═" * 70)
        
        # Texte original
        original_text = texts[i]
        print(f"📄 Texte original: '{original_text[:100]}{'...' if len(original_text) > 100 else ''}'")
        print(f"📏 Longueur originale: {len(original_text)} caractères, {len(original_text.split())} mots")
        
        # Données tokenisées
        input_ids = encodings['input_ids'][i].numpy()
        attention_mask = encodings['attention_mask'][i].numpy()
        
        print(f"\n🔢 Input IDs: {input_ids[:20]}{'...' if len(input_ids) > 20 else ''}")
        print(f"👁️ Attention mask: {attention_mask[:20]}{'...' if len(attention_mask) > 20 else ''}")
        
        # Décodage
        decoded_text = tokenizer.decode(input_ids, skip_special_tokens=False)
        print(f"🔄 Texte décodé: '{decoded_text}'")
        
        # Statistiques
        non_padding_tokens = np.sum(attention_mask)
        padding_tokens = len(attention_mask) - non_padding_tokens
        
        print(f"\n📊 Statistiques:")
        print(f"   🎯 Tokens réels: {non_padding_tokens}")
        print(f"   📝 Tokens de padding: {padding_tokens}")
        print(f"   📏 Longueur totale: {len(input_ids)}")
        print(f"   📊 Ratio padding: {padding_tokens/len(input_ids)*100:.1f}%")
        
        # Analyse des tokens spéciaux
        special_tokens_found = []
        if tokenizer.cls_token_id in input_ids:
            special_tokens_found.append("[CLS]")
        if tokenizer.sep_token_id in input_ids:
            special_tokens_found.append("[SEP]")
        if tokenizer.pad_token_id in input_ids:
            special_tokens_found.append("[PAD]")
        
        print(f"   🎯 Tokens spéciaux: {', '.join(special_tokens_found)}")

# Analyser quelques exemples
analyser_tokenisation(train_texts, train_encodings, tokenizer, 2)

print("\n💡 Points clés de la tokenisation:")
print("   🔤 Chaque texte commence par [CLS] et finit par [SEP]")
print("   📝 Les textes courts sont complétés avec [PAD]")
print("   ✂️ Les textes longs sont tronqués à MAX_LENGTH")
print("   👁️ L'attention mask aide le modèle à ignorer le padding")

## 📦 Création des Datasets TensorFlow

Maintenant que nos textes sont tokenisés, créons les datasets TensorFlow optimisés pour l'entraînement.

### 🎯 Pourquoi tf.data.Dataset ?

- **Performance optimisée** : Lecture et préprocessing efficaces
- **Batching automatique** : Groupement des exemples
- **Prefetching** : Chargement en parallèle de l'entraînement
- **Mélange des données** : Évite la mémorisation de l'ordre

In [ ]:
# 📦 Création des datasets TensorFlow
print("📦 Création des datasets TensorFlow optimisés...\n")

# Dataset d'entraînement
print("🏗️ Construction du dataset d'entraînement...")
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': train_encodings['input_ids'],
        'attention_mask': train_encodings['attention_mask']
    },
    train_labels
))

# Optimisations pour l'entraînement
train_dataset = train_dataset.shuffle(buffer_size=1000)  # Mélanger les données
train_dataset = train_dataset.batch(BATCH_SIZE)          # Créer des batches
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE) # Préchargement parallèle

print("✅ Dataset d'entraînement créé")

# Dataset de validation
print("🎯 Construction du dataset de validation...")
test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': test_encodings['input_ids'],
        'attention_mask': test_encodings['attention_mask']
    },
    test_labels
))

# Optimisations pour la validation (pas de shuffle)
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

print("✅ Dataset de validation créé")

print(f"\n📊 Configuration des datasets:")
print(f"   📈 Batch size: {BATCH_SIZE}")
print(f"   🔀 Shuffle train: Oui (buffer=1000)")
print(f"   🎯 Shuffle test: Non (ordre préservé)")
print(f"   ⚡ Prefetch: AUTOTUNE (parallélisation)")

### 🔍 Exploration des Datasets

Vérifions que nos datasets sont correctement configurés :

In [ ]:
# 🔍 Exploration des datasets créés
def explorer_dataset(dataset, nom_dataset, num_batches=1):
    """
    Explore la structure d'un dataset TensorFlow
    """
    print(f"🔍 === EXPLORATION DU DATASET {nom_dataset.upper()} ===")
    
    for batch_num, (features, labels) in enumerate(dataset.take(num_batches)):
        print(f"\n📦 Batch {batch_num + 1}:")
        print("─" * 50)
        
        # Structure des features
        print(f"🔧 Structure des features:")
        for key, value in features.items():
            print(f"   • {key}: {value.shape} (dtype: {value.dtype})")
        
        # Structure des labels
        print(f"🏷️ Labels: {labels.shape} (dtype: {labels.dtype})")
        
        # Exemple du premier élément
        print(f"\n📝 Premier exemple du batch:")
        input_ids_sample = features['input_ids'][0].numpy()
        attention_mask_sample = features['attention_mask'][0].numpy()
        label_sample = labels[0].numpy()
        
        print(f"   🔢 Input IDs (premiers 10): {input_ids_sample[:10]}")
        print(f"   👁️ Attention mask (premiers 10): {attention_mask_sample[:10]}")
        print(f"   🏷️ Label: {label_sample} ({'Positif' if label_sample == 1 else 'Négatif'})")
        
        # Décodage de l'exemple
        decoded_example = tokenizer.decode(input_ids_sample, skip_special_tokens=True)
        print(f"   📄 Texte décodé: '{decoded_example[:100]}{'...' if len(decoded_example) > 100 else ''}'")
        
        # Statistiques du batch
        print(f"\n📊 Statistiques du batch:")
        print(f"   📏 Taille du batch: {len(labels)}")
        print(f"   😊 Exemples positifs: {tf.reduce_sum(tf.cast(labels, tf.int32)).numpy()}")
        print(f"   😞 Exemples négatifs: {len(labels) - tf.reduce_sum(tf.cast(labels, tf.int32)).numpy()}")
        
        # Analyse du padding
        padding_ratios = []
        for i in range(len(features['attention_mask'])):
            mask = features['attention_mask'][i].numpy()
            real_tokens = np.sum(mask)
            padding_ratio = (len(mask) - real_tokens) / len(mask) * 100
            padding_ratios.append(padding_ratio)
        
        avg_padding = np.mean(padding_ratios)
        print(f"   📝 Padding moyen: {avg_padding:.1f}%")

# Explorer les datasets
explorer_dataset(train_dataset, "Entraînement", 1)
explorer_dataset(test_dataset, "Validation", 1)

print("\n✅ Datasets TensorFlow prêts pour l'entraînement !")
print("\n🎯 Prochaine étape: Construction du modèle DistilBERT")

## 🧠 Construction du Modèle avec DistilBERT

Maintenant que nos données sont tokenisées, construisons notre modèle de classification basé sur DistilBERT pré-entraîné.

### 🎯 Qu'est-ce que DistilBERT ?

**DistilBERT** est une version "distillée" de BERT :

- **66 millions de paramètres** (vs 110M pour BERT-base)
- **6 couches Transformer** (vs 12 pour BERT)
- **Même vocabulaire** que BERT (30k tokens)
- **97% des performances** de BERT
- **Pré-entraîné** sur de vastes corpus de texte anglais

### 🏗️ Architecture de notre Modèle

Notre modèle aura cette structure :
1. **DistilBERT pré-entraîné** (extraction de features)
2. **Pooling du token [CLS]** (représentation globale)
3. **Dropout** (régularisation)
4. **Dense layer** (classification binaire)

In [ ]:
# 🧠 Chargement du modèle DistilBERT pré-entraîné
print("🧠 Chargement de DistilBERT pré-entraîné...")
print("(Cela peut prendre quelques minutes la première fois)\n")

# Chargement depuis Hugging Face
distilbert_model = TFAutoModel.from_pretrained(
    'distilbert-base-uncased',
    return_dict=True  # Retour sous forme de dictionnaire
)

print("✅ DistilBERT chargé avec succès !")
print(f"📊 Nombre de paramètres: {distilbert_model.num_parameters():,}")
print(f"🏗️ Nombre de couches: 6 couches Transformer")
print(f"🎯 Dimension des embeddings: 768")
print(f"🔤 Vocabulaire: {tokenizer.vocab_size:,} tokens")

print(f"\n🎓 Avantages de DistilBERT:")
print(f"   ⚡ 60% plus rapide que BERT")
print(f"   💾 40% moins de mémoire")
print(f"   🎯 Performances quasi-identiques")
print(f"   📱 Adapté aux applications en production")

### 🏗️ Construction du Modèle Complet

Créons maintenant l'architecture complète qui combine DistilBERT avec nos couches de classification :

In [ ]:
# 🏗️ Fonction de création du modèle complet
def create_sentiment_model(distilbert_model, dropout_rate=0.3):
    """
    Crée un modèle de classification de sentiment basé sur DistilBERT
    
    Architecture :
    1. DistilBERT pré-entraîné (couche de base)
    2. Extraction du token [CLS] (représentation globale)
    3. Dropout pour régularisation
    4. Dense layer pour classification binaire
    
    Args:
        distilbert_model: Modèle DistilBERT pré-entraîné
        dropout_rate: Taux de dropout (défaut: 0.3)
    
    Returns:
        model: Modèle Keras complet
        base_model: Modèle DistilBERT (pour le fine-tuning)
    """
    print("🏗️ Construction de l'architecture complète...")
    
    # Couches d'entrée
    input_ids = tf.keras.layers.Input(
        shape=(MAX_LENGTH,), 
        dtype=tf.int32, 
        name='input_ids'
    )
    attention_mask = tf.keras.layers.Input(
        shape=(MAX_LENGTH,), 
        dtype=tf.int32, 
        name='attention_mask'
    )
    
    print("   ✅ Couches d'entrée créées")
    
    # DistilBERT pré-entraîné
    distilbert_output = distilbert_model(
        input_ids=input_ids,
        attention_mask=attention_mask
    )
    
    # Extraction des embeddings de séquence
    sequence_output = distilbert_output.last_hidden_state  # Shape: (batch_size, seq_len, 768)
    print("   ✅ DistilBERT intégré")
    
    # Pooling : extraction du token [CLS] (première position)
    cls_token = sequence_output[:, 0, :]  # Shape: (batch_size, 768)
    print("   ✅ Token [CLS] extrait")
    
    # Couche de dropout pour la régularisation
    dropout_output = tf.keras.layers.Dropout(dropout_rate, name='dropout')(cls_token)
    print(f"   ✅ Dropout ajouté (rate={dropout_rate})")
    
    # Couche de classification finale
    predictions = tf.keras.layers.Dense(
        1, 
        activation='sigmoid',  # Sigmoid pour classification binaire
        name='classifier'
    )(dropout_output)
    print("   ✅ Couche de classification ajoutée")
    
    # Création du modèle final
    model = tf.keras.Model(
        inputs=[input_ids, attention_mask],
        outputs=predictions,
        name='DistilBERT_Sentiment_Classifier'
    )
    
    print("\n✅ Modèle construit avec succès !")
    
    return model, distilbert_model

# Construction du modèle
model, base_model = create_sentiment_model(distilbert_model)

# Informations sur le modèle
total_params = sum(tf.size(var).numpy() for var in model.trainable_variables)
print(f"\n📊 Statistiques du modèle:")
print(f"   🔢 Paramètres totaux: {total_params:,}")
print(f"   🎯 Type de classification: Binaire (sentiment)")
print(f"   📏 Longueur d'entrée: {MAX_LENGTH} tokens")
print(f"   🎲 Dropout: 0.3 (30% des neurones désactivés)")

### 📋 Résumé de l'Architecture

Visualisons l'architecture de notre modèle :

In [ ]:
# 📋 Affichage du résumé du modèle
print("📋 Architecture détaillée du modèle:")
print("═" * 80)
model.summary()

print("\n🎓 Explication de l'architecture:")
print("   📥 Entrées:")
print("      • input_ids: Tokens numériques (batch_size, 256)")
print("      • attention_mask: Masque pour ignorer padding (batch_size, 256)")
print("   🧠 DistilBERT:")
print("      • 6 couches Transformer")
print("      • Multi-head attention (12 têtes)")
print("      • Embeddings contextuels (768 dimensions)")
print("   🎯 Classification:")
print("      • Token [CLS] → représentation globale")
print("      • Dropout → régularisation")
print("      • Dense(1) + Sigmoid → probabilité sentiment")

### 🎨 Visualisation de l'Architecture

Créons un diagramme pour mieux comprendre le flux de données :

In [ ]:
# 🎨 Visualisation de l'architecture du modèle
def visualiser_architecture_distilbert():
    """
    Crée un diagramme explicatif de l'architecture DistilBERT
    """
    fig, ax = plt.subplots(1, 1, figsize=(16, 12))
    
    # Définir les composants et leurs positions
    components = [
        {"name": "Texte d'entrée\n'This movie is amazing!'\n(Critique de film)", "pos": (1, 10), "color": "lightblue", "frozen": False},
        {"name": "Tokenizer DistilBERT\n[CLS] this movie is amazing [SEP]\n+ attention_mask", "pos": (1, 8.5), "color": "lightyellow", "frozen": False},
        {"name": "Input IDs\n[101, 2023, 3185, 2003, 6429, 102]\n+ Attention Mask", "pos": (1, 7), "color": "lightgray", "frozen": False},
        {"name": "DistilBERT\n6 Couches Transformer\n🔒 PRÉ-ENTRAÎNÉ", "pos": (1, 5.5), "color": "lightcoral", "frozen": True},
        {"name": "Sequence Output\n(batch, 256, 768)\nEmbeddings contextuels", "pos": (1, 4), "color": "lightgreen", "frozen": True},
        {"name": "Token [CLS] Extraction\n(batch, 768)\nReprésentation globale", "pos": (1, 2.5), "color": "lightyellow", "frozen": False},
        {"name": "Dropout(0.3)\n🎲 Régularisation", "pos": (1, 1), "color": "lightpink", "frozen": False},
        {"name": "Dense(1) + Sigmoid\n🎯 Classification Binaire\nProbabilité [0, 1]", "pos": (1, -0.5), "color": "lightsteelblue", "frozen": False}
    ]
    
    # Dessiner les composants
    for i, comp in enumerate(components):
        x, y = comp["pos"]
        
        # Style différent pour les couches pré-entraînées
        if comp["frozen"]:
            bbox_props = dict(boxstyle="round,pad=0.3", facecolor=comp["color"], 
                            edgecolor="red", linewidth=2, linestyle="--")
        else:
            bbox_props = dict(boxstyle="round,pad=0.3", facecolor=comp["color"], 
                            edgecolor="black", linewidth=1)
        
        ax.text(x, y, comp["name"], ha="center", va="center", 
               fontsize=11, fontweight="bold", bbox=bbox_props)
        
        # Flèches entre composants
        if i < len(components) - 1:
            next_y = components[i+1]["pos"][1]
            ax.annotate("", xy=(x, next_y + 0.4), xytext=(x, y - 0.4),
                       arrowprops=dict(arrowstyle="->", lw=2, color="darkblue"))
    
    # Annotations explicatives
    ax.text(4, 8, "💭 Préprocessing\n• Tokenisation WordPiece\n• Ajout tokens spéciaux\n• Padding/Truncation\n• Attention mask", 
           fontsize=10, bbox=dict(boxstyle="round,pad=0.5", facecolor="wheat", alpha=0.8))
    
    ax.text(4, 5, "🧠 DistilBERT Core\n• 6 couches Transformer\n• 12 têtes d'attention\n• 66M paramètres\n• Pré-entraîné sur Wikipedia", 
           fontsize=10, bbox=dict(boxstyle="round,pad=0.5", facecolor="lightcyan", alpha=0.8))
    
    ax.text(4, 1.5, "🎯 Classification\n• Token [CLS] = résumé\n• Dropout évite overfitting\n• Sigmoid → probabilité\n• > 0.5 = Positif", 
           fontsize=10, bbox=dict(boxstyle="round,pad=0.5", facecolor="lavender", alpha=0.8))
    
    # Informations techniques sur le côté
    ax.text(7, 3, f"⚙️ Configuration Technique\n\n📊 Batch size: {BATCH_SIZE}\n📏 Max length: {MAX_LENGTH}\n🎯 Learning rate: {LEARNING_RATE}\n⏳ Epochs: {EPOCHS}\n\n💾 Mémoire GPU: ~2-4GB\n⚡ Vitesse: ~60% plus rapide que BERT\n🎯 Performance: ~97% de BERT", 
           fontsize=9, bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.9))
    
    # Configuration des axes
    ax.set_xlim(-0.5, 9)
    ax.set_ylim(-2, 11)
    ax.set_title("🧠 Architecture DistilBERT pour Classification de Sentiment", fontsize=16, fontweight="bold", pad=20)
    ax.axis('off')
    
    # Légende
    legend_elements = [
        plt.Rectangle((0, 0), 1, 1, facecolor='lightcoral', edgecolor='red', linestyle='--', label='Composants pré-entraînés'),
        plt.Rectangle((0, 0), 1, 1, facecolor='lightsteelblue', edgecolor='black', label='Nouvelles couches')
    ]
    ax.legend(handles=legend_elements, loc='upper left')
    
    plt.tight_layout()
    plt.show()

print("🎨 Visualisation de l'architecture DistilBERT:")
visualiser_architecture_distilbert()

print("\n💡 Points clés de l'architecture:")
print("   🧠 DistilBERT: Modèle pré-entraîné riche en connaissances linguistiques")
print("   🎯 Token [CLS]: Représentation globale de tout le texte")
print("   🎲 Dropout: Évite le surapprentissage sur la nouvelle tâche")
print("   📈 Sigmoid: Sortie entre 0 et 1 (probabilité de sentiment positif)")

### 🔧 Test du Modèle

Testons que notre modèle fonctionne correctement avant l'entraînement :

In [ ]:
# 🔧 Test du modèle avant entraînement
def tester_modele_avant_entrainement(model, test_dataset):
    """
    Teste que le modèle fonctionne correctement avant l'entraînement
    """
    print("🔧 Test du modèle avant entraînement...\n")
    
    # Prendre un petit batch pour le test
    for batch_features, batch_labels in test_dataset.take(1):
        print(f"📦 Batch de test:")
        print(f"   📏 Taille: {len(batch_labels)} exemples")
        print(f"   📊 Input IDs shape: {batch_features['input_ids'].shape}")
        print(f"   👁️ Attention mask shape: {batch_features['attention_mask'].shape}")
        print(f"   🏷️ Labels shape: {batch_labels.shape}")
        
        # Forward pass (prédiction)
        print(f"\n🔮 Test de prédiction...")
        predictions = model([
            batch_features['input_ids'],
            batch_features['attention_mask']
        ], training=False)
        
        print(f"✅ Prédictions générées !")
        print(f"   📊 Shape des prédictions: {predictions.shape}")
        print(f"   📈 Valeurs min/max: {tf.reduce_min(predictions):.4f} / {tf.reduce_max(predictions):.4f}")
        print(f"   🎯 Moyenne: {tf.reduce_mean(predictions):.4f}")
        
        # Analyse des premières prédictions
        print(f"\n📝 Premières prédictions (avant entraînement):")
        for i in range(min(5, len(predictions))):
            pred_prob = predictions[i].numpy()[0]
            true_label = batch_labels[i].numpy()
            pred_sentiment = "Positif" if pred_prob > 0.5 else "Négatif"
            true_sentiment = "Positif" if true_label == 1 else "Négatif"
            
            print(f"   Exemple {i+1}: Prédit={pred_sentiment} ({pred_prob:.3f}), Vrai={true_sentiment}")
        
        break
    
    print(f"\n💡 Observations:")
    print(f"   🎲 Prédictions aléatoires (modèle non entraîné)")
    print(f"   📊 Valeurs entre 0 et 1 (sigmoid fonctionne)")
    print(f"   ✅ Architecture compatible avec nos données")
    print(f"   🚀 Prêt pour l'entraînement !")

# Tester le modèle
tester_modele_avant_entrainement(model, test_dataset)

## ⚙️ Compilation du Modèle

Maintenant que notre architecture est construite, configurons l'entraînement :

### 🎯 Configuration des Hyperparamètres

Pour le fine-tuning de DistilBERT, nous utiliserons :

- **Loss Function** : `BinaryCrossentropy` (classification binaire)
- **Optimiseur** : `Adam` avec learning rate adaptatif
- **Métriques** : `Accuracy` et `Precision/Recall`
- **Learning Rate** : Plus faible pour préserver les poids pré-entraînés

In [ ]:
# ⚙️ Configuration de l'optimiseur et des métriques
print("⚙️ Configuration de l'entraînement...\n")

# Optimiseur Adam avec learning rate adapté au fine-tuning
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,  # 2e-5 - plus faible pour préserver les poids
    epsilon=1e-08,               # Stabilité numérique
    clipnorm=1.0                 # Gradient clipping
)

print(f"🎯 Optimiseur configuré:")
print(f"   📊 Type: Adam")
print(f"   📈 Learning rate: {LEARNING_RATE}")
print(f"   ✂️ Gradient clipping: 1.0")
print(f"   🎯 Epsilon: 1e-08")

# Loss function pour classification binaire
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

print(f"\n📉 Loss function:")
print(f"   🎯 Type: Binary Crossentropy")
print(f"   📊 From logits: False (on utilise sigmoid)")

# Métriques de suivi
metrics = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

print(f"\n📊 Métriques de suivi:")
for metric in metrics:
    print(f"   • {metric.name}")

print(f"\n💡 Pourquoi ces choix ?")
print(f"   📈 Learning rate faible: Préserver les connaissances pré-entraînées")
print(f"   ✂️ Gradient clipping: Éviter l'explosion des gradients")
print(f"   📊 Multiple métriques: Vue complète des performances")

### 🔧 Compilation du Modèle

Compilons maintenant notre modèle avec ces paramètres :

In [ ]:
# 🔧 Compilation du modèle
print("🔧 Compilation du modèle...\n")

model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=metrics
)

print("✅ Modèle compilé avec succès !")

# Vérification de la configuration
print(f"\n📋 Configuration finale:")
print(f"   🎯 Tâche: Classification binaire de sentiment")
print(f"   🧠 Architecture: DistilBERT + Classification head")
print(f"   📊 Paramètres: {sum(tf.size(var).numpy() for var in model.trainable_variables):,}")
print(f"   ⚡ Optimiseur: {optimizer.__class__.__name__}")
print(f"   📉 Loss: {loss_fn.__class__.__name__}")
print(f"   📈 Métriques: {[m.name for m in metrics]}")

print(f"\n🚀 Modèle prêt pour l'entraînement !")

## 📞 Configuration des Callbacks

Les callbacks nous permettent de surveiller et contrôler l'entraînement :

### 🎯 Callbacks Utiles pour le Fine-tuning

- **EarlyStopping** : Arrêt automatique si pas d'amélioration
- **ReduceLROnPlateau** : Réduction du learning rate si stagnation
- **ModelCheckpoint** : Sauvegarde du meilleur modèle
- **CSVLogger** : Enregistrement des métriques

In [ ]:
# 📞 Configuration des callbacks pour l'entraînement
print("📞 Configuration des callbacks...\n")

# Early Stopping - arrêt si pas d'amélioration
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',     # Métrique à surveiller
    patience=3,                 # Nombre d'epochs sans amélioration
    restore_best_weights=True,  # Restaurer les meilleurs poids
    verbose=1
)

print("🛑 Early Stopping configuré:")
print(f"   📊 Surveille: validation accuracy")
print(f"   ⏳ Patience: 3 epochs")
print(f"   🔄 Restaure meilleurs poids: Oui")

# Reduce Learning Rate - diminution si stagnation
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',     # Métrique à surveiller
    factor=0.5,             # Facteur de réduction (LR = LR * 0.5)
    patience=2,             # Patience avant réduction
    min_lr=1e-7,           # Learning rate minimum
    verbose=1
)

print(f"\n📉 Reduce LR configuré:")
print(f"   📊 Surveille: validation loss")
print(f"   📈 Facteur: 0.5 (divise par 2)")
print(f"   ⏳ Patience: 2 epochs")
print(f"   🔻 LR minimum: 1e-7")

# Model Checkpoint - sauvegarde du meilleur modèle
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_distilbert_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

print(f"\n💾 Model Checkpoint configuré:")
print(f"   📁 Fichier: best_distilbert_model.h5")
print(f"   📊 Surveille: validation accuracy")
print(f"   💾 Sauve: Modèle complet")

# CSV Logger - enregistrement des métriques
csv_logger = tf.keras.callbacks.CSVLogger(
    'training_log_distilbert.csv',
    append=True
)

print(f"\n📝 CSV Logger configuré:")
print(f"   📁 Fichier: training_log_distilbert.csv")
print(f"   📊 Contenu: Toutes les métriques par epoch")

# Liste complète des callbacks
callbacks = [early_stopping, reduce_lr, checkpoint, csv_logger]

print(f"\n✅ {len(callbacks)} callbacks configurés:")
for i, cb in enumerate(callbacks, 1):
    print(f"   {i}. {cb.__class__.__name__}")

print(f"\n💡 Avantages des callbacks:")
print(f"   🛑 Évite le surapprentissage (Early Stopping)")
print(f"   📈 Optimise l'apprentissage (ReduceLR)")
print(f"   💾 Sauvegarde automatique (Checkpoint)")
print(f"   📊 Traçabilité complète (CSV Logger)")

## 🚀 Lancement de l'Entraînement (Fine-tuning)

C'est le moment de démarrer le fine-tuning de DistilBERT !

### 🎯 Stratégie de Fine-tuning

Nous utilisons une approche **End-to-End** :
- Toutes les couches sont entraînables dès le début
- Learning rate faible pour préserver les connaissances
- Régularisation avec dropout
- Surveillance avec callbacks

In [ ]:
# 🚀 Préparation au lancement de l'entraînement
print("🚀 Préparation de l'entraînement...\n")

# Vérification des paramètres d'entraînement
print("📊 Paramètres d'entraînement:")
print(f"   📈 Epochs: {EPOCHS}")
print(f"   📦 Batch size: {BATCH_SIZE}")
print(f"   📏 Sequence length: {MAX_LENGTH}")
print(f"   🎯 Learning rate: {LEARNING_RATE}")

# Calcul du nombre de steps par epoch
steps_per_epoch = len(list(train_dataset.as_numpy_iterator()))
validation_steps = len(list(test_dataset.as_numpy_iterator()))


print(f"\n📊 Configuration des données:")
print(f"   🏋️ Steps par epoch: {steps_per_epoch}")
print(f"   ✅ Steps de validation: {validation_steps}")
print(f"   ⏱️ Temps estimé: ~{EPOCHS * 3}-{EPOCHS * 5} minutes")

# Estimation mémoire GPU
print(f"\n💾 Utilisation mémoire estimée:")
print(f"   🖥️ GPU: ~2-4 GB")
print(f"   🧠 Paramètres: {sum(tf.size(var).numpy() for var in model.trainable_variables):,}")

print(f"\n🎯 Objectif: Classifier le sentiment (Positif/Négatif)")
print(f"📊 Métrique principale: Validation Accuracy")
print(f"\n🚀 Lancement de l'entraînement !")
print("=" * 60)

In [ ]:
# 🚀 Entraînement du modèle
print("🔥 DÉBUT DE L'ENTRAÎNEMENT\n")

# Enregistrer l'heure de début
import time
start_time = time.time()

# Entraînement avec fit()
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
    callbacks=callbacks,
    verbose=1
)

# Calcul du temps d'entraînement
end_time = time.time()
training_time = end_time - start_time

print(f"\n🎉 ENTRAÎNEMENT TERMINÉ !")
print(f"⏱️ Temps total: {training_time/60:.1f} minutes")
print(f"⚡ Temps par epoch: {training_time/EPOCHS:.1f} secondes")

# Sauvegarde de l'historique
import pickle
with open('training_history_distilbert.pkl', 'wb') as f:
    pickle.dump(history.history, f)

print(f"\n💾 Historique sauvegardé dans 'training_history_distilbert.pkl'")
print(f"📊 Modèle sauvegardé automatiquement par ModelCheckpoint")

### 📊 Résumé de l'Entraînement

Analysons rapidement les résultats de l'entraînement :

In [ ]:
# 📊 Analyse rapide des résultats d'entraînement
print("📊 RÉSUMÉ DE L'ENTRAÎNEMENT\n")

# Récupération des métriques finales
final_train_accuracy = history.history['accuracy'][-1]
final_val_accuracy = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

print(f"🎯 Performances finales:")
print(f"   📈 Accuracy entraînement: {final_train_accuracy:.4f} ({final_train_accuracy*100:.2f}%)")
print(f"   ✅ Accuracy validation: {final_val_accuracy:.4f} ({final_val_accuracy*100:.2f}%)")
print(f"   📉 Loss entraînement: {final_train_loss:.4f}")
print(f"   📊 Loss validation: {final_val_loss:.4f}")

# Analyse de l'overfitting
overfitting_gap = final_train_accuracy - final_val_accuracy
print(f"\n🔍 Analyse:")
if overfitting_gap < 0.05:
    print(f"   ✅ Bon équilibre (gap: {overfitting_gap:.4f})")
elif overfitting_gap < 0.10:
    print(f"   ⚠️ Léger overfitting (gap: {overfitting_gap:.4f})")
else:
    print(f"   🚨 Overfitting détecté (gap: {overfitting_gap:.4f})")

# Meilleure époch
best_epoch = history.history['val_accuracy'].index(max(history.history['val_accuracy'])) + 1
best_val_acc = max(history.history['val_accuracy'])

print(f"\n🏆 Meilleure performance:")
print(f"   📊 Époque: {best_epoch}/{EPOCHS}")
print(f"   🎯 Validation accuracy: {best_val_acc:.4f} ({best_val_acc*100:.2f}%)")

# Évaluation de la performance
if best_val_acc > 0.90:
    print(f"   🌟 Excellente performance !")
elif best_val_acc > 0.85:
    print(f"   👍 Très bonne performance !")
elif best_val_acc > 0.80:
    print(f"   ✅ Bonne performance")
else:
    print(f"   ⚠️ Performance à améliorer")

print(f"\n💡 Prochaine étape: Visualisation détaillée des courbes d'apprentissage")

## 📈 Visualisation des Courbes d'Entraînement

Analysons en détail l'évolution de notre modèle pendant l'entraînement.

### 🎯 Métriques à Analyser

- **Loss** : Évolution de l'erreur (diminution souhaitée)
- **Accuracy** : Précision de classification (augmentation souhaitée)
- **Precision/Recall** : Métriques détaillées
- **Écart train/validation** : Détection du surapprentissage

In [ ]:
# 📈 Fonction de visualisation des courbes d'entraînement
def visualiser_courbes_entrainement(history):
    """
    Crée des graphiques détaillés des métriques d'entraînement
    """
    print("📈 Création des visualisations...\n")
    
    # Configuration de la figure
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('📊 Évolution des Métriques - Fine-tuning DistilBERT', fontsize=16, fontweight='bold')
    
    # Couleurs pour les graphiques
    colors = {'train': '#2E86AB', 'val': '#A23B72'}
    
    # 1. Loss (Perte)
    ax1 = axes[0, 0]
    epochs_range = range(1, len(history.history['loss']) + 1)
    
    ax1.plot(epochs_range, history.history['loss'], 
             color=colors['train'], marker='o', linewidth=2, label='Entraînement')
    ax1.plot(epochs_range, history.history['val_loss'], 
             color=colors['val'], marker='s', linewidth=2, label='Validation')
    
    ax1.set_title('📉 Évolution de la Loss', fontweight='bold')
    ax1.set_xlabel('Époque')
    ax1.set_ylabel('Binary Crossentropy Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Annotation du minimum
    min_val_loss = min(history.history['val_loss'])
    min_epoch = history.history['val_loss'].index(min_val_loss) + 1
    ax1.annotate(f'Min: {min_val_loss:.4f}\n(Ép. {min_epoch})', 
                xy=(min_epoch, min_val_loss), xytext=(min_epoch+0.5, min_val_loss+0.01),
                arrowprops=dict(arrowstyle='->', color='red'), fontsize=9)
    
    # 2. Accuracy (Précision)
    ax2 = axes[0, 1]
    
    ax2.plot(epochs_range, [acc*100 for acc in history.history['accuracy']], 
             color=colors['train'], marker='o', linewidth=2, label='Entraînement')
    ax2.plot(epochs_range, [acc*100 for acc in history.history['val_accuracy']], 
             color=colors['val'], marker='s', linewidth=2, label='Validation')
    
    ax2.set_title('📊 Évolution de l\'Accuracy', fontweight='bold')
    ax2.set_xlabel('Époque')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Annotation du maximum
    max_val_acc = max(history.history['val_accuracy'])
    max_epoch = history.history['val_accuracy'].index(max_val_acc) + 1
    ax2.annotate(f'Max: {max_val_acc*100:.2f}%\n(Ép. {max_epoch})', 
                xy=(max_epoch, max_val_acc*100), 
                xytext=(max_epoch+0.5, max_val_acc*100-2),
                arrowprops=dict(arrowstyle='->', color='green'), fontsize=9)
    
    # 3. Precision
    ax3 = axes[1, 0]
    
    ax3.plot(epochs_range, [p*100 for p in history.history['precision']], 
             color=colors['train'], marker='o', linewidth=2, label='Entraînement')
    ax3.plot(epochs_range, [p*100 for p in history.history['val_precision']], 
             color=colors['val'], marker='s', linewidth=2, label='Validation')
    
    ax3.set_title('🎯 Évolution de la Precision', fontweight='bold')
    ax3.set_xlabel('Époque')
    ax3.set_ylabel('Precision (%)')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Recall
    ax4 = axes[1, 1]
    
    ax4.plot(epochs_range, [r*100 for r in history.history['recall']], 
             color=colors['train'], marker='o', linewidth=2, label='Entraînement')
    ax4.plot(epochs_range, [r*100 for r in history.history['val_recall']], 
             color=colors['val'], marker='s', linewidth=2, label='Validation')
    
    ax4.set_title('🔍 Évolution du Recall', fontweight='bold')
    ax4.set_xlabel('Époque')
    ax4.set_ylabel('Recall (%)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Graphiques des métriques générés !")

# Création des visualisations
visualiser_courbes_entrainement(history)

### 📊 Analyse Détaillée des Résultats

Créons un tableau récapitulatif des performances :

In [ ]:
# 📊 Analyse statistique détaillée
def analyser_performances_entrainement(history):
    """
    Analyse complète des performances d'entraînement
    """
    print("📊 ANALYSE DÉTAILLÉE DES PERFORMANCES\n")
    
    # Extraction des métriques
    metrics = {
        'Loss': {
            'train': history.history['loss'],
            'val': history.history['val_loss']
        },
        'Accuracy': {
            'train': history.history['accuracy'],
            'val': history.history['val_accuracy']
        },
        'Precision': {
            'train': history.history['precision'],
            'val': history.history['val_precision']
        },
        'Recall': {
            'train': history.history['recall'],
            'val': history.history['val_recall']
        }
    }
    
    # Création du tableau récapitulatif
    import pandas as pd
    
    results = []
    for metric_name, values in metrics.items():
        train_final = values['train'][-1]
        val_final = values['val'][-1]
        
        if metric_name == 'Loss':
            train_best = min(values['train'])
            val_best = min(values['val'])
            best_epoch = values['val'].index(val_best) + 1
        else:
            train_best = max(values['train'])
            val_best = max(values['val'])
            best_epoch = values['val'].index(val_best) + 1
        
        results.append({
            'Métrique': metric_name,
            'Train Final': f"{train_final:.4f}",
            'Val Final': f"{val_final:.4f}",
            'Meilleur Val': f"{val_best:.4f}",
            'Époque Opt.': best_epoch,
            'Gap Train-Val': f"{abs(train_final - val_final):.4f}"
        })
    
    df = pd.DataFrame(results)
    print("📋 Tableau récapitulatif:")
    print(df.to_string(index=False))
    
    # Analyse de convergence
    print(f"\n🔍 Analyse de convergence:")
    
    val_acc = metrics['Accuracy']['val']
    if len(val_acc) >= 3:
        last_3_epochs = val_acc[-3:]
        is_stable = max(last_3_epochs) - min(last_3_epochs) < 0.01
        
        if is_stable:
            print(f"   ✅ Convergence stable (variation < 1%)")
        else:
            print(f"   📈 Modèle encore en apprentissage")
    
    # Détection d'overfitting
    train_acc_final = metrics['Accuracy']['train'][-1]
    val_acc_final = metrics['Accuracy']['val'][-1]
    overfitting_gap = train_acc_final - val_acc_final
    
    print(f"\n🎯 Analyse de l'overfitting:")
    if overfitting_gap < 0.02:
        print(f"   ✅ Pas d'overfitting (gap: {overfitting_gap:.4f})")
    elif overfitting_gap < 0.05:
        print(f"   ⚠️ Overfitting léger (gap: {overfitting_gap:.4f})")
    else:
        print(f"   🚨 Overfitting significatif (gap: {overfitting_gap:.4f})")
    
    # F1-Score approximatif
    precision_final = metrics['Precision']['val'][-1]
    recall_final = metrics['Recall']['val'][-1]
    f1_score = 2 * (precision_final * recall_final) / (precision_final + recall_final)
    
    print(f"\n🏆 Score F1 estimé: {f1_score:.4f} ({f1_score*100:.2f}%)")
    
    # Recommandations
    print(f"\n💡 Recommandations:")
    if val_acc_final > 0.90:
        print(f"   🌟 Excellente performance ! Modèle prêt pour la production")
    elif val_acc_final > 0.85:
        print(f"   👍 Très bonne performance, peut être déployé")
    elif val_acc_final > 0.80:
        print(f"   ✅ Performance satisfaisante")
        if overfitting_gap > 0.05:
            print(f"   📝 Conseil: Augmenter la régularisation (dropout)")
    else:
        print(f"   ⚠️ Performance à améliorer")
        print(f"   📝 Conseils: Plus d'epochs, ajuster learning rate, ou plus de données")

# Lancer l'analyse
analyser_performances_entrainement(history)

## 🎯 Évaluation des Performances

Maintenant que notre modèle est entraîné, évaluons ses performances sur l'ensemble de test.

### 📊 Métriques d'Évaluation

Pour une classification binaire de sentiment, nous analyserons :

- **Accuracy** : Pourcentage de prédictions correctes
- **Precision** : Qualité des prédictions positives
- **Recall** : Capacité à détecter les vrais positifs
- **F1-Score** : Moyenne harmonique de precision et recall
- **Matrice de confusion** : Répartition détaillée des erreurs

In [ ]:
# 🎯 Évaluation sur l'ensemble de test
print("🎯 ÉVALUATION SUR L'ENSEMBLE DE TEST\n")

# Chargement du meilleur modèle (sauvegardé par ModelCheckpoint)
print("📂 Chargement du meilleur modèle...")
try:
    best_model = tf.keras.models.load_model('best_distilbert_model.h5')
    print("✅ Meilleur modèle chargé depuis ModelCheckpoint")
except:
    best_model = model  # Utiliser le modèle actuel si pas de sauvegarde
    print("⚠️ Utilisation du modèle actuel (pas de checkpoint trouvé)")

# Évaluation complète
print(f"\n🔬 Évaluation en cours...")
test_results = best_model.evaluate(test_dataset, verbose=1)

# Récupération des métriques
test_loss = test_results[0]
test_accuracy = test_results[1]
test_precision = test_results[2]
test_recall = test_results[3]

# Calcul du F1-Score
f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print(f"\n📊 RÉSULTATS SUR L'ENSEMBLE DE TEST:")
print(f"═" * 50)
print(f"🎯 Accuracy:  {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"🎯 Precision: {test_precision:.4f} ({test_precision*100:.2f}%)")
print(f"🎯 Recall:    {test_recall:.4f} ({test_recall*100:.2f}%)")
print(f"🏆 F1-Score:  {f1_score:.4f} ({f1_score*100:.2f}%)")
print(f"📉 Loss:      {test_loss:.4f}")
print(f"═" * 50)

# Interprétation des résultats
print(f"\n💭 Interprétation:")
if test_accuracy > 0.90:
    print(f"   🌟 Excellente performance ! Modèle prêt pour la production")
elif test_accuracy > 0.85:
    print(f"   🎉 Très bonne performance ! Modèle fiable")
elif test_accuracy > 0.80:
    print(f"   ✅ Bonne performance, utilisable en pratique")
elif test_accuracy > 0.75:
    print(f"   👍 Performance correcte, peut être améliorée")
else:
    print(f"   ⚠️ Performance insuffisante, nécessite optimisation")

# Analyse équilibre Precision/Recall
balance = abs(test_precision - test_recall)
if balance < 0.05:
    print(f"   ⚖️ Bon équilibre Precision/Recall (diff: {balance:.3f})")
else:
    print(f"   ⚖️ Déséquilibre Precision/Recall (diff: {balance:.3f})")
    if test_precision > test_recall:
        print(f"      → Modèle conservateur (peu de faux positifs)")
    else:
        print(f"      → Modèle permissif (peu de faux négatifs)")

### 🔍 Matrice de Confusion

Analysons en détail les types d'erreurs commises par notre modèle :

In [ ]:
# 🔍 Génération des prédictions pour la matrice de confusion
print("🔍 Génération des prédictions pour analyse détaillée...\n")

# Récupération des vraies étiquettes et prédictions
y_true = []
y_pred = []
y_pred_proba = []

print("📊 Collecte des prédictions...")
for batch_features, batch_labels in test_dataset:
    # Prédictions
    predictions = best_model([
        batch_features['input_ids'],
        batch_features['attention_mask']
    ], training=False)
    
    # Conversion en listes
    y_true.extend(batch_labels.numpy())
    y_pred_proba.extend(predictions.numpy().flatten())
    y_pred.extend((predictions.numpy() > 0.5).astype(int).flatten())

print(f"✅ {len(y_true)} prédictions collectées")

# Conversion en arrays numpy
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_pred_proba = np.array(y_pred_proba)

print(f"📊 Distribution des vraies étiquettes:")
print(f"   Négatives (0): {np.sum(y_true == 0)} ({np.sum(y_true == 0)/len(y_true)*100:.1f}%)")
print(f"   Positives (1): {np.sum(y_true == 1)} ({np.sum(y_true == 1)/len(y_true)*100:.1f}%)")

print(f"\n🎯 Distribution des prédictions:")
print(f"   Négatives (0): {np.sum(y_pred == 0)} ({np.sum(y_pred == 0)/len(y_pred)*100:.1f}%)")
print(f"   Positives (1): {np.sum(y_pred == 1)} ({np.sum(y_pred == 1)/len(y_pred)*100:.1f}%)")

In [ ]:
# 📈 Création de la matrice de confusion et visualisations
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

def visualiser_matrice_confusion(y_true, y_pred, y_pred_proba):
    """
    Crée une visualisation complète de la matrice de confusion
    """
    print("📈 Création de la matrice de confusion...\n")
    
    # Calcul de la matrice
    cm = confusion_matrix(y_true, y_pred)
    
    # Configuration de la figure
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # 1. Matrice de confusion
    ax1 = axes[0]
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Négatif', 'Positif'],
                yticklabels=['Négatif', 'Positif'], ax=ax1)
    ax1.set_title('🔍 Matrice de Confusion', fontweight='bold')
    ax1.set_xlabel('Prédiction')
    ax1.set_ylabel('Vérité')
    
    # Ajout des pourcentages
    total = cm.sum()
    for i in range(2):
        for j in range(2):
            percentage = cm[i, j] / total * 100
            ax1.text(j + 0.5, i + 0.7, f'({percentage:.1f}%)', 
                    ha='center', va='center', fontsize=10, color='gray')
    
    # 2. Distribution des probabilités prédites
    ax2 = axes[1]
    
    # Séparer par classe réelle
    neg_probs = y_pred_proba[y_true == 0]
    pos_probs = y_pred_proba[y_true == 1]
    
    ax2.hist(neg_probs, bins=30, alpha=0.7, label='Vrais Négatifs', color='lightcoral')
    ax2.hist(pos_probs, bins=30, alpha=0.7, label='Vrais Positifs', color='lightblue')
    ax2.axvline(x=0.5, color='red', linestyle='--', label='Seuil (0.5)')
    
    ax2.set_title('📊 Distribution des Probabilités', fontweight='bold')
    ax2.set_xlabel('Probabilité prédite')
    ax2.set_ylabel('Fréquence')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Analyse des erreurs
    ax3 = axes[2]
    
    # Types d'erreurs
    tn, fp, fn, tp = cm.ravel()
    
    categories = ['Vrais\nNégatifs', 'Faux\nPositifs', 'Faux\nNégatifs', 'Vrais\nPositifs']
    values = [tn, fp, fn, tp]
    colors = ['lightgreen', 'lightcoral', 'lightsalmon', 'lightblue']
    
    bars = ax3.bar(categories, values, color=colors)
    ax3.set_title('📊 Répartition des Prédictions', fontweight='bold')
    ax3.set_ylabel('Nombre d\'exemples')
    
    # Ajout des valeurs sur les barres
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'{value}\n({value/total*100:.1f}%)', 
                ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Analyse textuelle
    print(f"📋 Analyse détaillée de la matrice:")
    print(f"   ✅ Vrais Négatifs (TN):  {tn:4d} ({tn/total*100:.1f}%)")
    print(f"   ❌ Faux Positifs (FP):   {fp:4d} ({fp/total*100:.1f}%) - Erreur Type I")
    print(f"   ❌ Faux Négatifs (FN):   {fn:4d} ({fn/total*100:.1f}%) - Erreur Type II")
    print(f"   ✅ Vrais Positifs (TP):  {tp:4d} ({tp/total*100:.1f}%)")
    
    print(f"\n🎯 Interprétation des erreurs:")
    if fp > fn:
        print(f"   📊 Plus de Faux Positifs: Le modèle tend à sur-classifier comme positif")
        print(f"   💡 Conseil: Augmenter le seuil de décision (> 0.5)")
    elif fn > fp:
        print(f"   📊 Plus de Faux Négatifs: Le modèle tend à sous-classifier comme positif")
        print(f"   💡 Conseil: Diminuer le seuil de décision (< 0.5)")
    else:
        print(f"   ⚖️ Erreurs équilibrées: Bon seuil de décision")

# Génération des visualisations
visualiser_matrice_confusion(y_true, y_pred, y_pred_proba)

### 📝 Rapport de Classification

Générons un rapport détaillé avec toutes les métriques :

In [ ]:
# 📝 Rapport de classification détaillé
print("📝 RAPPORT DE CLASSIFICATION DÉTAILLÉ\n")

# Rapport scikit-learn
class_report = classification_report(y_true, y_pred, 
                                   target_names=['Négatif', 'Positif'],
                                   output_dict=True)

print("📊 Métriques par classe:")
print("═" * 70)
print(f"{'Classe':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'Support':<10}")
print("═" * 70)

for class_name in ['Négatif', 'Positif']:
    metrics = class_report[class_name]
    print(f"{class_name:<10} {metrics['precision']:<10.4f} {metrics['recall']:<10.4f} "
          f"{metrics['f1-score']:<10.4f} {int(metrics['support']):<10}")

print("═" * 70)
print(f"{'Macro Avg':<10} {class_report['macro avg']['precision']:<10.4f} "
      f"{class_report['macro avg']['recall']:<10.4f} "
      f"{class_report['macro avg']['f1-score']:<10.4f} "
      f"{int(class_report['macro avg']['support']):<10}")
print(f"{'Weighted Avg':<10} {class_report['weighted avg']['precision']:<10.4f} "
      f"{class_report['weighted avg']['recall']:<10.4f} "
      f"{class_report['weighted avg']['f1-score']:<10.4f} "
      f"{int(class_report['weighted avg']['support']):<10}")

# Analyse de la qualité globale
overall_f1 = class_report['weighted avg']['f1-score']
print(f"\n🏆 Score F1 Global: {overall_f1:.4f} ({overall_f1*100:.2f}%)")

print(f"\n💭 Évaluation de la qualité:")
if overall_f1 > 0.90:
    print(f"   🌟 Qualité exceptionnelle - Modèle de production")
elif overall_f1 > 0.85:
    print(f"   🎉 Excellente qualité - Prêt pour déploiement")
elif overall_f1 > 0.80:
    print(f"   ✅ Bonne qualité - Utilisable en pratique")
elif overall_f1 > 0.75:
    print(f"   👍 Qualité correcte - Améliorations possibles")
else:
    print(f"   ⚠️ Qualité insuffisante - Optimisation nécessaire")

# Comparaison avec un baseline
baseline_accuracy = max(np.sum(y_true == 0), np.sum(y_true == 1)) / len(y_true)
improvement = test_accuracy - baseline_accuracy

print(f"\n📊 Comparaison avec baseline:")
print(f"   🎯 Accuracy modèle: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"   📊 Baseline (classe majoritaire): {baseline_accuracy:.4f} ({baseline_accuracy*100:.2f}%)")
print(f"   📈 Amélioration: +{improvement:.4f} (+{improvement*100:.2f} points)")

if improvement > 0.20:
    print(f"   🚀 Amélioration exceptionnelle !")
elif improvement > 0.10:
    print(f"   🎉 Excellente amélioration !")
elif improvement > 0.05:
    print(f"   ✅ Bonne amélioration")
else:
    print(f"   ⚠️ Amélioration modeste")

print(f"\n💡 Le modèle DistilBERT apporte une valeur significative par rapport à une approche naïve !")

## 🔮 Fonction de Prédiction Interactive

Créons une fonction pratique pour tester notre modèle sur de nouvelles phrases !

### 🎯 Objectifs de cette Section

- **Fonction de prédiction** simple et réutilisable
- **Interface interactive** pour tester des phrases
- **Analyse de confiance** des prédictions
- **Exemples pratiques** avec différents types de textes
- **Conseils d'interprétation** des résultats

In [ ]:
# 🔮 Fonction de prédiction de sentiment
def predire_sentiment(texte, modele=None, tokenizer=tokenizer, max_length=MAX_LENGTH, verbose=True):
    """
    Prédit le sentiment d'un texte donné
    
    Args:
        texte (str): Texte à analyser
        modele: Modèle à utiliser (par défaut le meilleur modèle)
        tokenizer: Tokenizer DistilBERT
        max_length (int): Longueur maximale de séquence
        verbose (bool): Affichage détaillé
    
    Returns:
        dict: Résultats de prédiction
    """
    # Utiliser le meilleur modèle par défaut
    if modele is None:
        try:
            modele = tf.keras.models.load_model('best_distilbert_model.h5')
        except:
            modele = best_model  # Fallback sur le modèle actuel
    
    if verbose:
        print(f"🔮 Analyse du sentiment pour: \"{texte}\"\n")
    
    # Tokenisation du texte
    tokens = tokenizer(
        texte,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )
    
    if verbose:
        print(f"📝 Tokenisation:")
        decoded_tokens = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
        # Afficher seulement les tokens non-padding
        real_tokens = [tok for tok in decoded_tokens if tok != '[PAD]'][:20]  # Limiter l'affichage
        print(f"   🔤 Tokens: {' '.join(real_tokens)}{'...' if len(real_tokens) == 20 else ''}")
        print(f"   📏 Longueur: {len([tok for tok in decoded_tokens if tok != '[PAD]'])} tokens")
    
    # Prédiction
    prediction = modele([
        tokens['input_ids'],
        tokens['attention_mask']
    ], training=False)
    
    # Extraction des résultats
    probabilite = float(prediction[0][0])
    sentiment = "Positif" if probabilite > 0.5 else "Négatif"
    confiance = max(probabilite, 1 - probabilite)
    
    if verbose:
        print(f"\n🎯 Résultats:")
        print(f"   📊 Sentiment: {sentiment}")
        print(f"   📈 Probabilité: {probabilite:.4f}")
        print(f"   🎯 Confiance: {confiance:.4f} ({confiance*100:.1f}%)")
        
        # Interprétation de la confiance
        if confiance > 0.9:
            print(f"   💪 Très haute confiance - Prédiction très fiable")
        elif confiance > 0.8:
            print(f"   ✅ Haute confiance - Prédiction fiable")
        elif confiance > 0.7:
            print(f"   👍 Confiance modérée - Prédiction raisonnable")
        elif confiance > 0.6:
            print(f"   ⚠️ Faible confiance - Prédiction incertaine")
        else:
            print(f"   🤔 Très faible confiance - Texte ambigu")
    
    return {
        'texte': texte,
        'sentiment': sentiment,
        'probabilite': probabilite,
        'confiance': confiance,
        'tokens': len([tok for tok in tokenizer.convert_ids_to_tokens(tokens['input_ids'][0]) if tok != '[PAD]'])
    }

print("✅ Fonction de prédiction créée !")
print("🎯 Utilisez: predire_sentiment('Votre texte ici')")

### 🧪 Tests sur des Exemples Variés

Testons notre modèle sur différents types de textes :

In [ ]:
# 🧪 Tests sur des exemples variés
exemples_tests = [
    # Exemples clairement positifs
    "This movie is absolutely amazing! I loved every minute of it.",
    "Outstanding performance! One of the best films I've ever seen.",
    "Brilliant cinematography and excellent acting. Highly recommended!",
    
    # Exemples clairement négatifs
    "This movie is terrible. Complete waste of time.",
    "Awful acting and boring plot. I fell asleep watching it.",
    "One of the worst movies ever made. Absolutely horrible.",
    
    # Exemples neutres/ambigus
    "The movie was okay. Not great, not terrible.",
    "It has some good moments but also some flaws.",
    "Average film with decent acting but predictable plot.",
    
    # Exemples avec sarcasme/ironie
    "Oh great, another generic superhero movie. Just what we needed.",
    "Sure, spending 3 hours watching paint dry would be more exciting."
]

print("🧪 TESTS SUR EXEMPLES VARIÉS\n")
print("═" * 80)

resultats_tests = []

for i, exemple in enumerate(exemples_tests, 1):
    print(f"\n📝 Test {i}/{len(exemples_tests)}:")
    print("─" * 60)
    
    resultat = predire_sentiment(exemple, verbose=True)
    resultats_tests.append(resultat)
    
    print("─" * 60)

print(f"\n✅ Tests terminés sur {len(exemples_tests)} exemples !")

### 📊 Analyse des Résultats de Test

Analysons les performances sur nos exemples de test :

In [ ]:
# 📊 Analyse des résultats de test
def analyser_resultats_tests(resultats):
    """
    Analyse les résultats des tests de prédiction
    """
    print("📊 ANALYSE DES RÉSULTATS DE TEST\n")
    
    # Création d'un DataFrame pour l'analyse
    import pandas as pd
    
    df_results = pd.DataFrame([
        {
            'Texte': r['texte'][:50] + '...' if len(r['texte']) > 50 else r['texte'],
            'Sentiment': r['sentiment'],
            'Probabilité': f"{r['probabilite']:.3f}",
            'Confiance': f"{r['confiance']:.3f}",
            'Tokens': r['tokens']
        } for r in resultats
    ])
    
    print("📋 Tableau récapitulatif:")
    print(df_results.to_string(index=False))
    
    # Statistiques générales
    nb_positifs = sum(1 for r in resultats if r['sentiment'] == 'Positif')
    nb_negatifs = len(resultats) - nb_positifs
    confiance_moyenne = sum(r['confiance'] for r in resultats) / len(resultats)
    
    print(f"\n📊 Statistiques:")
    print(f"   📈 Prédictions positives: {nb_positifs}/{len(resultats)} ({nb_positifs/len(resultats)*100:.1f}%)")
    print(f"   📉 Prédictions négatives: {nb_negatifs}/{len(resultats)} ({nb_negatifs/len(resultats)*100:.1f}%)")
    print(f"   🎯 Confiance moyenne: {confiance_moyenne:.3f} ({confiance_moyenne*100:.1f}%)")
    
    # Analyse de la confiance
    haute_confiance = sum(1 for r in resultats if r['confiance'] > 0.8)
    faible_confiance = sum(1 for r in resultats if r['confiance'] < 0.6)
    
    print(f"\n🎯 Analyse de confiance:")
    print(f"   💪 Haute confiance (>80%): {haute_confiance}/{len(resultats)} ({haute_confiance/len(resultats)*100:.1f}%)")
    print(f"   ⚠️ Faible confiance (<60%): {faible_confiance}/{len(resultats)} ({faible_confiance/len(resultats)*100:.1f}%)")
    
    # Recommandations
    print(f"\n💡 Observations:")
    if confiance_moyenne > 0.8:
        print(f"   ✅ Excellent niveau de confiance général")
    elif confiance_moyenne > 0.7:
        print(f"   👍 Bon niveau de confiance")
    else:
        print(f"   ⚠️ Niveau de confiance à améliorer")
    
    if faible_confiance > 0:
        print(f"   🔍 {faible_confiance} exemple(s) avec faible confiance - textes probablement ambigus")
    
    return df_results

# Analyse des résultats
df_analyse = analyser_resultats_tests(resultats_tests)

## 🎓 Conclusion et Récapitulatif

Félicitations ! Vous avez complété avec succès un projet complet de **fine-tuning DistilBERT** pour l'analyse de sentiment.

### 🏆 Ce que Vous Avez Accompli

Au cours de ce notebook pédagogique, vous avez maîtrisé :

#### 🔍 **Exploration et Préparation des Données**
- Chargement du dataset IMDB (50,000 critiques de films)
- Analyse statistique et visualisation de la distribution
- Préparation des données d'entraînement, validation et test
- Gestion de datasets déséquilibrés

#### 🔤 **Tokenisation et Preprocessing NLP**
- Utilisation du tokenizer DistilBERT WordPiece
- Gestion des tokens spéciaux (`[CLS]`, `[SEP]`, `[PAD]`)
- Création d'attention masks pour ignorer le padding
- Optimisation des séquences avec padding et truncation

#### 🧠 **Architecture et Modélisation**
- Chargement d'un modèle DistilBERT pré-entraîné
- Construction d'une tête de classification personnalisée
- Intégration du pooling et de la régularisation (dropout)
- Compréhension du transfer learning en NLP

### 💡 Concepts Clés

#### 🧠 **Architecture Transformer**
- **Self-attention** : Mécanisme permettant au modèle de "faire attention" aux mots importants
- **Multi-head attention** : Plusieurs têtes d'attention pour capturer différents types de relations
- **Positional encodings** : Comment le modèle comprend l'ordre des mots
- **Layer normalization** : Technique de normalisation pour stabiliser l'entraînement

#### 🔤 **Tokenisation Avancée**
- **WordPiece** : Algorithme de tokenisation sous-mot intelligent
- **Vocabulary size** : Impact de la taille du vocabulaire (30,522 tokens pour DistilBERT)
- **Out-of-vocabulary handling** : Gestion des mots non vus pendant l'entraînement
- **Special tokens** : Rôle crucial des tokens `[CLS]`, `[SEP]`, `[MASK]`, `[PAD]`

#### 🎯 **Transfer Learning en NLP**
- **Pre-training** : Entraînement sur de vastes corpus de texte non labellisé
- **Fine-tuning** : Adaptation à une tâche spécifique avec peu de données
- **Feature extraction vs Fine-tuning** : Différentes stratégies d'adaptation
- **Learning rate scheduling** : Importance d'un LR faible pour préserver les connaissances